<a href="https://colab.research.google.com/github/bhavuk912/Lane-detection/blob/main/lane%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from google.colab import files
from IPython.display import HTML, display, clear_output
from base64 import b64encode
from google.colab.patches import cv2_imshow
from collections import deque

# Global variables for lane smoothing
left_lane_history = deque(maxlen=10)  # Store last 10 frames
right_lane_history = deque(maxlen=10)

def process_frame(image):
    """Process a single frame to detect lanes"""
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Apply Gaussian blur to reduce noise
    blur_gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Define a more focused region of interest (ROI) mask
    mask = np.zeros_like(edges)
    mask_color = 255
    imshape = image.shape

    # Narrower trapezoid focusing on road lanes
    vertices = np.array([[
        (int(imshape[1] * 0.1), imshape[0]),
        (int(imshape[1] * 0.4), int(imshape[0] * 0.65)),
        (int(imshape[1] * 0.6), int(imshape[0] * 0.65)),
        (int(imshape[1] * 0.9), imshape[0])
    ]], dtype=np.int32)

    # Apply the mask to edges
    cv2.fillPoly(mask, vertices, mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Apply Hough Line Transform
    rho = 2
    theta = np.pi / 180
    threshold = 25
    min_line_length = 60
    max_line_gap = 80

    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold,
                            np.array([]), min_line_length, max_line_gap)

    # Create a blank image to draw lines on
    line_image = np.zeros_like(image)

    if lines is not None:
        # Separate left and right lane lines based on slope
        left_lines = []
        right_lines = []

        for line in lines:
            x1, y1, x2, y2 = line[0]

            # Calculate slope
            if x2 - x1 == 0:
                continue
            slope = (y2 - y1) / (x2 - x1)

            # Calculate line length
            length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

            # Filter by length and slope
            if length < 40:
                continue

            # Strict slope filtering
            if -0.9 < slope < -0.5:  # Left lane
                left_lines.append(line[0])
            elif 0.5 < slope < 0.9:  # Right lane
                right_lines.append(line[0])

        # Average the lines
        left_lane = average_lines(left_lines, imshape)
        right_lane = average_lines(right_lines, imshape)

        # Apply temporal smoothing
        left_lane = smooth_lane(left_lane, left_lane_history)
        right_lane = smooth_lane(right_lane, right_lane_history)

        # Draw the lines
        if left_lane is not None:
            cv2.line(line_image, (left_lane[0], left_lane[1]),
                    (left_lane[2], left_lane[3]), (0, 255, 0), 10)

        if right_lane is not None:
            cv2.line(line_image, (right_lane[0], right_lane[1]),
                    (right_lane[2], right_lane[3]), (0, 255, 0), 10)

    # Combine the original image with the lane lines
    result = cv2.addWeighted(image, 0.8, line_image, 1, 0)

    return result

def average_lines(lines, imshape):
    """Average multiple line segments into a single line with weighted averaging"""
    if len(lines) == 0:
        return None

    # Calculate weighted average slope and intercept (weight by line length)
    slopes = []
    intercepts = []
    weights = []

    for line in lines:
        x1, y1, x2, y2 = line
        if x2 - x1 == 0:
            continue

        # Calculate line length as weight
        length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1

        slopes.append(slope)
        intercepts.append(intercept)
        weights.append(length)

    if len(slopes) == 0:
        return None

    # Weighted average
    weights = np.array(weights)
    avg_slope = np.average(slopes, weights=weights)
    avg_intercept = np.average(intercepts, weights=weights)

    # Calculate line endpoints
    y1 = imshape[0]  # Bottom of image
    y2 = int(imshape[0] * 0.6)  # Middle of region of interest

    x1 = int((y1 - avg_intercept) / avg_slope)
    x2 = int((y2 - avg_intercept) / avg_slope)

    return [x1, y1, x2, y2]

def smooth_lane(lane, lane_history):
    """Apply temporal smoothing to lane lines"""
    if lane is None:
        # If no lane detected, use the last known lane
        if len(lane_history) > 0:
            return lane_history[-1]
        return None

    # Add current lane to history
    lane_history.append(lane)

    # Return weighted average of historical lanes (more weight to recent frames)
    if len(lane_history) == 0:
        return lane

    # Create weights that favor recent frames
    weights = np.linspace(0.5, 1.0, len(lane_history))
    weights = weights / weights.sum()

    # Calculate weighted average of all coordinates
    avg_lane = np.zeros(4)
    for i, historical_lane in enumerate(lane_history):
        avg_lane += np.array(historical_lane) * weights[i]

    return avg_lane.astype(int).tolist()

def upload_video():
    """Upload video file in Google Colab"""
    print("Please select your video file...")
    uploaded = files.upload()

    # Get the uploaded file name
    video_filename = list(uploaded.keys())[0]
    print(f"Video '{video_filename}' uploaded successfully!")
    return video_filename

def display_video(video_path):
    """Display video in Colab notebook"""
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width=800 controls>
        <source src="{data_url}" type="video/mp4">
    </video>
    """)

def main():
    """Main function to process video"""
    global left_lane_history, right_lane_history

    # Reset lane history for new video
    left_lane_history.clear()
    right_lane_history.clear()

    # Upload video in Google Colab
    print("=== Stable Lane Detection System ===\n")
    video_filename = upload_video()

    # Open video file
    cap = cv2.VideoCapture(video_filename)

    if not cap.isOpened():
        print("Error: Could not open video file")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Create video writer to save output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output_lane_detection.mp4', fourcc, fps,
                         (frame_width, frame_height))

    print("Processing video with temporal smoothing...")

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print("\n✓ Video processing complete!")
            break

        frame_count += 1

        # Show progress every 10% of video
        if frame_count % max(1, total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"Progress: {progress:.0f}% ({frame_count}/{total_frames} frames)")

        # Convert BGR to RGB for processing
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame
        processed_frame = process_frame(frame_rgb)

        # Convert back to BGR for saving
        processed_frame_bgr = cv2.cvtColor(processed_frame, cv2.COLOR_RGB2BGR)

        # Write to output video
        out.write(processed_frame_bgr)

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print("\n" + "="*50)
    print("✓ Processing complete!")
    print("✓ Output saved as 'output_lane_detection.mp4'")
    print("="*50)

    # Display the output video in Colab
    print("\n📹 Your processed video is ready to watch:\n")
    display(display_video('output_lane_detection.mp4'))

    # Provide download option
    print("\n💾 To download the video, run the following in a new cell:")
    print("from google.colab import files")
    print("files.download('output_lane_detection.mp4')")

    return 'output_lane_detection.mp4'

# Run the main function
if __name__ == "__main__":
    output_file = main()

=== Stable Lane Detection System ===

Please select your video file...


Saving vecteezy_tbilisi-georgia-2021-street-view-from-driving-car-of_7534647.mov to vecteezy_tbilisi-georgia-2021-street-view-from-driving-car-of_7534647.mov
Video 'vecteezy_tbilisi-georgia-2021-street-view-from-driving-car-of_7534647.mov' uploaded successfully!
Processing video with temporal smoothing...
Progress: 10% (159/1590 frames)
Progress: 20% (318/1590 frames)
Progress: 30% (477/1590 frames)
Progress: 40% (636/1590 frames)
Progress: 50% (795/1590 frames)
Progress: 60% (954/1590 frames)
Progress: 70% (1113/1590 frames)
Progress: 80% (1272/1590 frames)
Progress: 90% (1431/1590 frames)
Progress: 100% (1590/1590 frames)

✓ Video processing complete!

✓ Processing complete!
✓ Output saved as 'output_lane_detection.mp4'

📹 Your processed video is ready to watch:

